# Environment Testing (after configuring the server for Detectron2)

1. Add Cuda, Conda, and OpenCV to PATH so its available inside Jupyter!

In [ ]:
import os

# Add Conda binary directory to the PATH
os.environ['PATH'] += ':/home/sirpoopsalot/miniconda/bin'

# Add CUDA binary directory to the PATH
os.environ['PATH'] += ':/usr/local/cuda/bin'

# # Add OpenCV binary directory to the PATH
# os.environ['PATH'] += ':/path/to/opencv/bin'  # Replace '/path/to/opencv/bin' with the actual path to your OpenCV binary directory

# Add CUDA library directory to LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

# test Conda
!conda info

# test Cuda
!nvcc --version

# # Test OpenCV
# import cv2

2. make sure we are working from within our new environment

In [ ]:
import sys
print(sys.executable)

3. Verify Torch is available by listing the version

In [ ]:
import torch
print(torch.__version__)

4. Verify that Dectron, PyTorch, and are available!

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

5. verify that we have GPU processing available to our model

   (this will make sure our drivers and libraires are correctly set)

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available. Using CPU.")


# Lets do some testing with Detectron2

1. configure the system with a basic setup!

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
#from detectron2.utils.logger import setup_logger
#setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# create work around for google colab cv2_imshow package
import matplotlib.pyplot as plt

def imshow(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

2. Lets get started by bringing in an image we want to evaluate!

In [ ]:
# Read an image
image_path = "/mnt/data/skyy-med/model_data/temp_test_data/people.jpg" 
my_new_image = cv2.imread(image_path)

# Set figure size
plt.figure(figsize=(10, 8))  # Adjust the values (width, height) as needed

# Display the image using matplotlib
plt.imshow(cv2.cvtColor(my_new_image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Hide axes
plt.show()

3. Next lets run the image againsta a pretrained model for testing Keypoint detection!

In [ ]:
# Inference with a keypoint detection model
cfg_keypoint = get_cfg()   # get a fresh new config
cfg_keypoint.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg_keypoint.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg_keypoint.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg_keypoint)
outputs = predictor(my_new_image)
v = Visualizer(my_new_image[:,:,::-1], MetadataCatalog.get(cfg_keypoint.DATASETS.TRAIN[0]), scale=3)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Set figure size
plt.figure(figsize=(10, 8))  # Adjust the values (width, height) as needed

imshow(out.get_image()[:, :, ::-1])

4. Next lets run the image againsta a pretrained model for Testing Instance segmentation!

In [ ]:
# Inference with instance segmentation
cfg_inst = get_cfg()
cfg_inst.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_inst.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo.  https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md
cfg_inst.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg_inst)
outputs = predictor(my_new_image)

v = Visualizer(my_new_image[:, :, ::-1], MetadataCatalog.get(cfg_inst.DATASETS.TRAIN[0]), scale=1.0)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Set figure size
plt.figure(figsize=(10, 8))  # Adjust the values (width, height) as needed

imshow(out.get_image()[:, :, ::-1])

5. Next lets run the image against a pretrained model for Testing Panoptic segmentation!

In [ ]:
# Inference with a panoptic segmentation model
cfg_pan = get_cfg()
cfg_pan.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg_pan.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg_pan)
panoptic_seg, segments_info = predictor(my_new_image)["panoptic_seg"]
v = Visualizer(my_new_image[:, :, ::-1], MetadataCatalog.get(cfg_pan.DATASETS.TRAIN[0]), scale=1.0)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)

# Set figure size
plt.figure(figsize=(10, 8))  # Adjust the values (width, height) as needed

imshow(out.get_image()[:, :, ::-1])

6. Next lets run the pretrained model on something it should not been trained to identify.
(Bring in a medical image)

In [ ]:
# Read an image
image_path = "/mnt/data/skyy-med/model_data/x-ray/FracAtlas/images/Fractured/IMG0002311.jpg" 
fracture_image = cv2.imread(image_path)

# Set figure size
plt.figure(figsize=(12, 10))  # Adjust the values (width, height) as needed

# Display the image using matplotlib
plt.imshow(cv2.cvtColor(fracture_image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Hide axes
plt.show()

7. Run the model on the medical image.

In [ ]:
sci_outputs = predictor(fracture_image)
sci_v = Visualizer(fracture_image[:, :, ::-1], MetadataCatalog.get(cfg_inst.DATASETS.TRAIN[0]))
sci_out = sci_v.draw_instance_predictions(sci_outputs["instances"].to("cpu"))

# Set figure size
plt.figure(figsize=(12, 10))  # Adjust the values (width, height) as needed

imshow(sci_out.get_image()[:, :, ::-1])